In [ ]:
config = {
    "basic": {
        "work_dir": "/fs2/home/ndsim10/DeepQT/DeepQTH/3_predict/predict/graphene450",
        "OLP_dir": "/fs2/home/ndsim10/DeepQT/DeepQTH/3_predict/predict/graphene450/cal_overlap",
        "system_name": "graphene450",
        "interface": "siesta",
        "input_file": "input.fdf",
        "trained_model_dir": [
            "/fs2/home/ndsim10/DeepQT/DeepQTH/2_train/trained_model/2025-10-16_16-02-14-deeph"
        ],
        "task": [1, 2, 3, 4, 5],
        "sparse_calc_config": "/fs2/home/ndsim10/DeepQT/DeepQTH/3_predict/predict/graphene450/band.json",
        "eigen_solver": "dense_py",
        "disable_cuda": False,
        "device": "cpu",
        "huge_structure": True,
        "restore_blocks_py": True,
    },
    "interpreter": {
        "python_interpreter": "~/miniconda3/envs/deeph-cpu/bin/python"
    },
    "graph": {
        "radius": 7.0, #graphene 7.0 Å, MoS2 8.0 Å, and silicon  9.0 Å
        "shortest_path_length": 5,
    }
}


In [ ]:
import os
import time
import subprocess as sp
import json

import sys
import os

sys.path.insert(0, "/fs2/home/ndsim10/DeepQT/DeepQTH/1_preprocess")
from get_data_from_siesta import get_data_from_siesta
from rotate import rotate_back
from get_rotate_coord import get_rc

sys.path.insert(0, "/fs2/home/ndsim10/DeepQT/DeepQTH/3_predict")
from pred_ham import predict, predict_with_grad

def main(config):

    work_dir = os.path.abspath(config["basic"]["work_dir"])
    OLP_dir = os.path.abspath(config["basic"]["OLP_dir"])
    system_name = os.path.abspath(config["basic"]["system_name"]).split('/')[-1]
    interface = config["basic"]["interface"] #siesta/transiesta
    input_file = config['basic']['input_file'] #input.fdf
    task = config["basic"]["task"] #[1, 2, 3, 4, 5]
    assert isinstance(task, list)
    eigen_solver = config["basic"]["eigen_solver"] #dense_py
    disable_cuda = config["basic"]["disable_cuda"] #False
    device = config["basic"]["device"] #cpu
    huge_structure = config["basic"]["huge_structure"] #True
    restore_blocks_py = config["basic"]["restore_blocks_py"] #True
    python_interpreter = config["interpreter"]["python_interpreter"] #python
    radius = config["graph"]["radius"] #9.0
    print("system_name = ",system_name)

    if 5 in task:
        if eigen_solver in ['dense_py']:
            assert python_interpreter, "Please specify python_interpreter to use Python code to calculate eigenpairs"
        else:
            raise ValueError(f"Unknown eigen_solver: {eigen_solver}")

    os.makedirs(work_dir, exist_ok=True)

    with open(os.path.join(work_dir, 'config.txt'), 'w') as f:
        json.dump(config, f, indent=2, ensure_ascii=False)

    if eigen_solver == 'dense_py':
        cmd5 = f"{python_interpreter} " \
               f"{os.path.join(os.getcwd(), 'dense_calc.py')} " \
               f"--input_dir {work_dir} --output_dir {work_dir} --config {config['basic']['sparse_calc_config']}"
    else:
        raise ValueError(f"Unknown eigen_solver: {eigen_solver}")


    print(f"\n~~~~~~~ 1.parse_Overlap\n")
    print(f"\n~~~~~~~ 2.get_local_coordinate\n")
    print(f"\n~~~~~~~ 3.get_pred_Hamiltonian\n")
    print(f"\n~~~~~~~ 4.rotate_back\n")
    print(f"\n~~~~~~~ 5.sparse_calc, command: \n{cmd5}\n")


    if 1 in task:
        begin = time.time()
        print(f"\n####### Begin 1.parse_Overlap")

        assert os.path.exists(os.path.join(OLP_dir, '{}.HSX'.format(system_name))), "Necessary files could not be found in OLP_dir"
        get_data_from_siesta(OLP_dir, work_dir, interface, input_file)
        assert os.path.exists(os.path.join(work_dir, "overlaps.h5"))
        assert os.path.exists(os.path.join(work_dir, "lat.dat"))
        assert os.path.exists(os.path.join(work_dir, "rlat.dat"))
        assert os.path.exists(os.path.join(work_dir, "site_positions.dat"))
        assert os.path.exists(os.path.join(work_dir, "orbital_types.dat"))
        assert os.path.exists(os.path.join(work_dir, "element.dat"))
        print('\n******* Finish 1.parse_Overlap, cost %d seconds\n' % (time.time() - begin))
    
    if 2 in task: #True
        begin = time.time()
        print(f"\n####### Begin 2.get_local_coordinate")
        get_rc(work_dir, work_dir, radius=radius) 
        assert os.path.exists(os.path.join(work_dir, "rc.h5"))
        print('\n******* Finish 2.get_local_coordinate, cost %d seconds\n' % (time.time() - begin))
    
    
    if 3 in task:
        begin = time.time()
        print(f"\n####### Begin 3.get_pred_Hamiltonian")
        trained_model_dir = config["basic"]["trained_model_dir"]
        if trained_model_dir[0] == '[' and trained_model_dir[-1] == ']':
            trained_model_dir = json.loads(trained_model_dir)
        print(trained_model_dir)
        
        predict(input_dir=work_dir, output_dir=work_dir, disable_cuda=disable_cuda, device=device,
                huge_structure=huge_structure, restore_blocks_py=restore_blocks_py,
                trained_model_dirs=trained_model_dir)
        if restore_blocks_py: #True
            assert os.path.exists(os.path.join(work_dir, "rh_pred.h5"))
        else:
            capture_output = sp.run(cmd3_post, shell=True, capture_output=False, encoding="utf-8")
            assert capture_output.returncode == 0
            assert os.path.exists(os.path.join(work_dir, "rh_pred.h5"))
        print('\n******* Finish 3.get_pred_Hamiltonian, cost %d seconds\n' % (time.time() - begin))
    
    
    if 4 in task:
        begin = time.time()
        print(f"\n####### Begin 4.rotate_back")
        rotate_back(input_dir=work_dir, output_dir=work_dir)
        assert os.path.exists(os.path.join(work_dir, "hamiltonians_pred.h5"))
        print('\n******* Finish 4.rotate_back, cost %d seconds\n' % (time.time() - begin))

    """
    if 5 in task:
        begin = time.time()
        print(f"\n####### Begin 5.sparse_calc")
        capture_output = sp.run(cmd5, shell=True, capture_output=False, encoding="utf-8")
        assert capture_output.returncode == 0
        if eigen_solver in ['sparse_jl']:
            assert os.path.exists(os.path.join(work_dir, "sparse_matrix.jld"))
        print('\n******* Finish 5.sparse_calc, cost %d seconds\n' % (time.time() - begin))
    """

if __name__ == '__main__':
    main(config)


In [ ]:
import sisl
from sisl.io import *
import numpy as np
import h5py

large_structure_path = "/fs2/home/ndsim10/DeepQT/DeepQTH/3_predict/predict/graphene450/cal_overlap/"
large_geom = sisl.get_sile(large_structure_path + "input.fdf").read_geometry()
hsx = hsxSileSiesta(large_structure_path + "graphene450.HSX")
large_structure_empty_hamiltonian = hsx.read_hamiltonian(geometry=large_geom)


predict_hamiltonian_file = "/fs2/home/ndsim10/DeepQT/DeepQTH/3_predict/predict/graphene450/hamiltonians_pred.h5"
predict_hamiltonian = {}
with h5py.File(predict_hamiltonian_file, "r") as fid:
    for key in fid.keys():
        H_ab = np.array(fid[key])
        processed_key = tuple(map(int, key[1:-1].split(',')))
        predict_hamiltonian[processed_key] = H_ab
overlaps_file = "/fs2/home/ndsim10/DeepQT/DeepQTH/3_predict/predict/graphene450/overlaps.h5"
overlaps = {}
with h5py.File(overlaps_file, "r") as fid:
    for key in fid.keys():
        # print(key)
        S_ab = np.array(fid[key])
        processed_key = tuple(map(int, key[1:-1].split(',')))
        overlaps[processed_key] = S_ab


def find_atom_sc(R, atom):
    # print(atom)
    asc_list = large_structure_empty_hamiltonian.auc2sc(atoms=atom)
    for sc_atom in asc_list:
        isc = list(large_structure_empty_hamiltonian.a2isc(atoms=sc_atom))
        # print(isc)
        if isc == R:
            # print("find the isc:", isc, sc_atom)
            return sc_atom

for key in predict_hamiltonian.keys():
    # print(key)
    hamiltonian_pred = predict_hamiltonian[key] 
    # print(hamiltonian_pred)
    if key in overlaps.keys():
        overlap = overlaps[key]
    else:
        overlap = np.zeros_like(hamiltonian_pred)
    # print(overlap)
    R = list(key[:3]) 
    atom_i = key[3]
    atom_j = key[4]

    sc_atom_j = find_atom_sc(R, atom_j)
    # print(sc_atom_j)
    
    center_atom_orbital_list = np.array(large_structure_empty_hamiltonian.a2o(atoms=atom_i, all=True))
    neighbor_atom_orbital_list = np.array(large_structure_empty_hamiltonian.a2o(atoms=sc_atom_j, all=True))
    # print(center_atom_orbital_list)
    # print(neighbor_atom_orbital_list)

    for m, row in enumerate(center_atom_orbital_list):
        for n, col in enumerate(neighbor_atom_orbital_list):
            large_structure_empty_hamiltonian.H[row, col] = hamiltonian_pred[m, n]

    # Hij = large_structure_empty_hamiltonian.sub(atoms=[atom_i, sc_atom_j]).tocsr().toarray()
    # print(Hij.shape)
    # print(Hij[9:,27:36])
    # break

    

In [ ]:
large_structure_empty_hamiltonian.shape

In [ ]:
large_structure_path = "/fs2/home/ndsim10/DeepH-final/graphene_example/work_dir/olp/graphene450/"
large_geom = sisl.get_sile(large_structure_path + "input.fdf").read_geometry()
hsx = hsxSileSiesta(large_structure_path + "graphene450.HSX")
H_one_step_scf = hsx.read_hamiltonian(geometry=large_geom)
H_one_step_scf.shape

In [ ]:
for a, b in large_structure_empty_hamiltonian.iter_nnz():
    print(large_structure_empty_hamiltonian[a,b], H_one_step_scf[a,b])
    if a == 1:
        break

In [ ]:
def find_atom_sc(R, atom):
    print(atom)
    asc_list = large_structure_empty_hamiltonian.auc2sc(atoms=atom)
    for sc_atom in asc_list:
        isc = list(large_structure_empty_hamiltonian.a2isc(atoms=sc_atom))
        # print(isc)
        if isc == R:
            print("find the isc:", isc, sc_atom)
            return sc_atom

sc_atom_j = find_atom_sc(R, atom_j)
print(sc_atom_j)

In [ ]:
H_block_matrix = dict()
S_block_matrix = dict()
seen = set()
for i, j in large_structure_empty_hamiltonian.iter_nnz():
    a_i = large_structure_empty_hamiltonian.o2a(orbitals=i, unique=True) # orbit i belongs to atom_1。
    b_j = large_structure_empty_hamiltonian.o2a(orbitals=j, unique=True)
    uc_b_j = large_structure_empty_hamiltonian.asc2uc(atoms=b_j)
    isc = large_structure_empty_hamiltonian.a2isc(atoms=b_j)
    key = '[{}, {}, {}, {}, {}]'.format(isc[0],isc[1],isc[2],a_i,uc_b_j)
    print(key)

    if i == 2:
        break
    
    # if key not in seen:
    #     # print(key)
    #     seen.add(key)
    #     H_ab = H.sub([a_i, b_j])
    #     H_ab_array = H_ab.tocsr().toarray()
    #     H_ab_matrix = H_ab_array[:9,9:18]
    #     S_ab = S.sub([a_i, b_j])
    #     S_ab_array = S_ab.tocsr().toarray()
    #     S_ab_matrix = S_ab_array[:9,9:18]
    #     #draw_sub_H(a_i, b_j, S_ab_matrix)
    #     H_block_matrix[key] = H_ab_matrix
    #     S_block_matrix[key] = S_ab_matrix
        
# with h5py.File(os.path.join(output_path, "hamiltonians.h5"), "w") as f:
#     for key in H_block_matrix.keys():
#         f[key] = H_block_matrix[key]

In [ ]:
geom = sisl.get_sile("/input.fdf").read_geometry()
hsx = hsxSileSiesta("/graphene450.HSX")
H = hsx.read_hamiltonian(geometry=geom)
S = hsx.read_overlap(geometry=geom)
print(H)
print(H.shape)
print(S.shape)

In [ ]:
bs = BandStructure(
    H,
    [[0,0,0], [0.5,0,0], [0.33333333,0.33333333,0], [0,0,0]],
    30,
    [r"Γ", r"M", r"K", r"Γ"]
)
lk, kt, kl = bs.lineark(True)
print(lk, kt, kl)
bs_eig = bs.apply.array.eigh()
# print(bs_eig)
print(bs_eig.shape)

In [ ]:
plt.figure(figsize=(6, 5))
plt.xlabel('Wave Vector k', fontsize=10)
plt.ylabel('Energy (eV)', fontsize=10)
plt.title('Band Structure', fontsize=10)

for i in range(bs_eig.shape[1]):
    if i == 0:
        plt.plot(lk, bs_eig[:, i] + 5.352835, color='blue', linewidth=1, label='Siesta')
    else:
        plt.plot(lk, bs_eig[:, i] + 5.352835, color='blue', linewidth=1)

    
plt.legend(loc='center', fontsize=10)

plt.ylim(-3, 3)
plt.xlim(kt[0], kt[-1])
plt.xticks(kt, kl)
plt.grid()
plt.show()

In [ ]:
from sisl.physics.electron import *

plt.figure(figsize=(6, 4))
bz = MonkhorstPack(H, [3,3,1])
E = np.linspace(-8,8,50)
eig = bz.apply.average.eigh()

dos = DOS(E, eig.real)
plt.plot(E, dos, color='blue', linewidth=1, label='Siesta')
plt.scatter(E, dos, marker='o',color='red', s=2, label='DeepQTH')
plt.xlabel(r"$E - E_F$ [eV]", fontsize=10)
plt.ylabel(r"DOS [1/eV]", fontsize=10)
# plt.grid()
plt.legend(loc='upper right', fontsize=10)
plt.show()